In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

from fastai import *
from fastai.torch_core import *

In [2]:
train_fname = Path('train.npz')
try:
    npzfile = np.load(train_fname)
    x = npzfile['x']
    y = npzfile['y']
    m = npzfile['m']
    max_items, max_atoms = x.shape[0], x.shape[-1]
    
    assert y.shape[1] == 2
except:
    t = pd.read_csv('train.csv')
    s = pd.read_csv('structures.csv')
    t['molecule_index'] = pd.factorize(t['molecule_name'])[0]
    t['type_index']     = pd.factorize(t['type'])[0]
    s = pd.concat([s,pd.get_dummies(s['atom'])], axis=1)

    max_items = 785836
    max_atoms = s.atom_index.max() + 1

    xyz   =['x', 'y', 'z']
    a_hot =['C','F','H','N','O']
    x = np.zeros((max_items,len(xyz)+len(a_hot),max_atoms), dtype=np.float32)
    y = np.zeros((max_items,2,max_atoms), dtype=np.float32)
    m = np.zeros((max_items,), dtype=np.int32)
    i = j = 0
    for (m_name, m_index) ,m_group in tqdm(t.groupby(['molecule_name', 'molecule_index'])):
        ss = s[s.molecule_name==m_name]
        n_atoms = len(ss)
        for a_name,a_group in m_group.groupby('atom_index_0'):
            y [i,:] = np.nan
            ref_a = ss[ss['atom_index']==a_name]
            x [i,:3] = 0.
            x [i,3:] = ref_a[a_hot].values.T
            x [i,:3,:n_atoms] = (ss[xyz].values-ref_a[xyz].values).T  # xyz 
            x [i,3:,:n_atoms] = ss[a_hot].T                           # a_hot
            y [i,0,a_group['atom_index_1']] = a_group['scalar_coupling_constant']
            y [i,1,a_group['atom_index_1']] = a_group['type_index']
            m[i] = m_index
            i+=1
        j += 1
    assert i == max_items
    np.savez(train_fname, x=x,y=y,m=m)

100%|██████████| 85003/85003 [2:45:15<00:00,  8.65it/s]  


In [3]:
y[0].T

array([[ 84.8076,   0.    ],
       [     nan,      nan],
       [-11.257 ,   1.    ],
       [-11.2548,   1.    ],
       ...,
       [     nan,      nan],
       [     nan,      nan],
       [     nan,      nan],
       [     nan,      nan]], dtype=float32)

In [4]:
x.shape,y.shape,m.shape

((785836, 8, 29), (785836, 2, 29), (785836,))

In [5]:
x[0].T

array([[-0.014849,  1.091835,  0.006025,  1.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ],
       [ 1.00958 ,  1.469782, -0.0017  ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ],
       [-0.542965,  1.453558, -0.87862 ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  1.      ,  0.      ,  0.      ]], dtype=float32)

In [6]:
y[0].T

array([[ 84.8076,   0.    ],
       [     nan,      nan],
       [-11.257 ,   1.    ],
       [-11.2548,   1.    ],
       ...,
       [     nan,      nan],
       [     nan,      nan],
       [     nan,      nan],
       [     nan,      nan]], dtype=float32)

In [7]:
m[0].T

0

In [8]:
# from https://github.com/fxia22/pointnet.pytorch/blob/master/pointnet/model.py
class STN3d(nn.Module):
    def __init__(self):a
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = False):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

class PointNetCls(nn.Module):
    def __init__(self, k=2, feature_transform=False):
        super(PointNetCls, self).__init__()
        self.feature_transform = feature_transform
        self.feat = PointNetfeat(global_feat=True, feature_transform=feature_transform)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.3)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1), trans, trans_feat


class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2, feature_transform=False):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feature_transform=feature_transform
        self.feat = PointNetfeat(global_feat=False, feature_transform=feature_transform)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans, trans_feat
    
class PointNetDenseReg(nn.Module):
    def __init__(self, k = 2, feature_transform=False):
        super(PointNetDenseReg, self).__init__()
        self.k = k
        self.feature_transform=feature_transform
        self.feat = PointNetfeat(global_feat=False, feature_transform=feature_transform)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = x.view(batchsize, n_pts, self.k)
        return x, trans, trans_feat

def feature_transform_regularizer(trans):
    d = trans.size()[1]
    batchsize = trans.size()[0]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
    return loss

IndentationError: unexpected indent (<ipython-input-8-22b3a15093fb>, line 4)

In [ ]:
from fastai.torch_core import *
sim_data = Tensor(x)
trans = STN3d()
out = trans(sim_data[:32,:3,:])
print('stn', out.size())
print('loss', feature_transform_regularizer(out))

In [ ]:
sim_data_64d = Tensor(torch.rand(32, 64, 2500))
trans = STNkd(k=64)
out = trans(sim_data_64d)
print('stn64d', out.size())
print('loss', feature_transform_regularizer(out))

In [ ]:
pointfeat = PointNetfeat(global_feat=True)
out, _, _ = pointfeat(sim_data[:32,:3,:])
print('global feat', out.size())

In [ ]:
pointfeat = PointNetfeat(global_feat=False)
out, _, _ = pointfeat(sim_data[:32,:3,:])
print('point feat', out.size())

In [ ]:
# we'll use this one for regression
seg = PointNetDenseReg(k = 1)
out, _, _ = seg(sim_data[:32,:3,:])
print('seg', out.size())
out[0], y[:32]

In [ ]:
list(seg.parameters())[-3].shape

In [ ]:
seg